In [122]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from math import sqrt
filepath = 'C:/Users/Anthony/Dropbox/Background Understanding/Data Science/CDIPS/CDIPS15_TeamCat/data/competition_data/'
files = [x for x in os.listdir(filepath)] #if os.path.isfile(os.path.join(filepath,x)) if you want to check isfile

## Merge files into train and test dataframes

merging 'bill_of_materials.csv' and 'tube.csv' with training set and test set.

In [123]:
billdf = pd.read_csv(os.path.join(filepath,'bill_of_materials.csv'),header=0)
tubedf = pd.read_csv(os.path.join(filepath,'tube.csv'),header=0)
traindf = pd.read_csv(os.path.join(filepath,'train_set.csv'), header = 0, parse_dates = ['quote_date'])
testdf = pd.read_csv(os.path.join(filepath,'test_set.csv'), header = 0, parse_dates = ['quote_date'])

In [124]:
train = pd.merge(traindf,billdf, on = 'tube_assembly_id')
test = pd.merge(testdf,billdf, on = 'tube_assembly_id')
train = pd.merge(train,tubedf, on = 'tube_assembly_id')
test = pd.merge(test,tubedf, on = 'tube_assembly_id')

## Clean `NaN` values

For component_id_1,2,3,... columns, I replaced `NaN` values with an empty string.

For quantity_1,2,3,... columns, I replaced `NaN` values with 0

In [125]:
for x in range(7,24,2):
    column = train.columns[x]
    train[column].replace(np.nan,0, regex=True, inplace= True)

for x in range(8,23,2):
    column = train.columns[x]
    train[column].replace(np.nan,'',regex=True, inplace= True)    
    
for x in range(7,24,2):
    column = test.columns[x]
    test[column].replace(np.nan,0,regex=True, inplace= True)
    
for x in range(8,23,2):
    column = test.columns[x]
    test[column].replace(np.nan,'', regex=True, inplace= True)     


## Dropping Columns

From bill_of_materials.csv, I dropped these columns from train and test:

- 'component_id_4', 'component_id_5', 'component_id_6', 'component_id_7', 'component_id_8'
- 'quantity_4','quantity_5', 'quantity_6', 'quantity_7', 'quantity_8'

From tube.csv, I dropped these columns from train and test:

- 'end_a_1x','end_x_1x','num_bracket','num_boss','other'

From test, I dropped:

- 'id','annual_usage','min_order_quantity'

From train, I dropped:

- 'annual_usage','min_order_quantity'

These choices were made based of "Initial Exploration" ipython notebook and Bharat's TrainTest01 updated ipython notebook. Most of these columns are essentially single valued or are useless (like 'id')

In [126]:
print train.columns
print "Number of columns", len(train.columns)

Index([u'tube_assembly_id', u'supplier', u'quote_date', u'annual_usage',
       u'min_order_quantity', u'bracket_pricing', u'quantity', u'cost',
       u'component_id_1', u'quantity_1', u'component_id_2', u'quantity_2',
       u'component_id_3', u'quantity_3', u'component_id_4', u'quantity_4',
       u'component_id_5', u'quantity_5', u'component_id_6', u'quantity_6',
       u'component_id_7', u'quantity_7', u'component_id_8', u'quantity_8',
       u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_1x', u'end_a_2x', u'end_x_1x', u'end_x_2x',
       u'end_a', u'end_x', u'num_boss', u'num_bracket', u'other'],
      dtype='object')
Number of columns 39


In [127]:
traindrop = ['annual_usage','min_order_quantity','component_id_4','component_id_5', 'component_id_6', 'component_id_7', 'component_id_8','quantity_4','quantity_5', 'quantity_6', 'quantity_7', 'quantity_8','end_a_1x','end_x_1x','num_bracket','num_boss','other']              
testdrop =  ['id','annual_usage','min_order_quantity','component_id_4','component_id_5', 'component_id_6', 'component_id_7', 'component_id_8','quantity_4','quantity_5', 'quantity_6', 'quantity_7', 'quantity_8','end_a_1x','end_x_1x','num_bracket','num_boss','other']
train = train.drop(traindrop, axis = 1)
test = test.drop(testdrop, axis = 1)
print 'Train Columns'
print
print train.columns
print "Number of columns", len(train.columns)

print '\n\n'

print 'Test Columns'
print
print test.columns
print "Number of columns", len(test.columns)

Train Columns

Index([u'tube_assembly_id', u'supplier', u'quote_date', u'bracket_pricing',
       u'quantity', u'cost', u'component_id_1', u'quantity_1',
       u'component_id_2', u'quantity_2', u'component_id_3', u'quantity_3',
       u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_2x', u'end_x_2x', u'end_a', u'end_x'],
      dtype='object')
Number of columns 22



Test Columns

Index([u'tube_assembly_id', u'supplier', u'quote_date', u'bracket_pricing',
       u'quantity', u'component_id_1', u'quantity_1', u'component_id_2',
       u'quantity_2', u'component_id_3', u'quantity_3', u'material_id',
       u'diameter', u'wall', u'length', u'num_bends', u'bend_radius',
       u'end_a_2x', u'end_x_2x', u'end_a', u'end_x'],
      dtype='object')
Number of columns 21


## Add year, month, and days column

Since we only care about 2011-2014, I referenced days from the 0 point 12/31/2010. So 01/01/2011 is 1, 01/02/2011 is 2, etc...

After that, I dropped the 'quote_date' column because it's now superfluous.

In [128]:
train['year'] = train['quote_date'].dt.year
train['month'] = train['quote_date'].dt.month
train['day'] = [date.days for date in train['quote_date'] - dt.date(2010,12,31)]

test['year'] = test['quote_date'].dt.year
test['month'] = test['quote_date'].dt.month
test['day'] = [date.days for date in test['quote_date'] - dt.date(2010,12,31)]

In [129]:
train = train[(train['bracket_pricing'] == 'Yes') & (train['supplier'] == 'S-0066') & train['year'].isin(range(2011,2015))]
test = test[(test['bracket_pricing'] == 'Yes') & (test['supplier'] == 'S-0066') & test['year'].isin(range(2011,2015))]

print "Train subsetting check"
print
print "Bracket Pricings:", train['bracket_pricing'].unique()
print "Suppliers:",  train['supplier'].unique()
print "Years:", train['year'].unique()
print "\n\n"
print "Test subsetting check"
print
print "Bracket Pricings:", test['bracket_pricing'].unique()
print "Suppliers:",  test['supplier'].unique()
print "Years:", test['year'].unique()
print "\n\n"

Train subsetting check

Bracket Pricings: ['Yes']
Suppliers: ['S-0066']
Years: [2013 2012 2011 2014]



Test subsetting check

Bracket Pricings: ['Yes']
Suppliers: ['S-0066']
Years: [2013 2014 2011 2012]





In [130]:
train = train.drop('quote_date', axis = 1)
test = test.drop('quote_date', axis = 1)

## Write to csv

In [131]:
train.to_csv('train.csv')
test.to_csv('test.csv')